# Notebook 4

[Download the notebook](notebook4.zip)

In this notebook we aim to model the evolution of "[Glacier du Mont Miné](https://www.google.com/maps/place/Mont+Min%C3%A9+Glacier/@46.0145075,7.4877889,24811m/data=!3m1!1e3!4m6!3m5!1s0x478f2db4b5d1a731:0x2c7d28e4d7f7bb26!8m2!3d46.014!4d7.54439!16s%2Fm%2F04n61lb?entry=ttu&g_ep=EgoyMDI0MTAyNy4wIKXMDSoASAFQAw%3D%3D,)" located in te South of Sion (Valais, Switzerland), from the end of the little ice age (about 1850) to now.

You may find on slides 28 to 30 some graphical [illustrations](https://www.unige.ch/forel/application/files/3116/5476/6042/F-Evolene-Paysagesglaciaires_compressed.pdf) of the retreat of the Mont Miné Glacier since the end of the Little Ice Age (LIA).

<!-- 
<img src="fig/mont_mine_2.png" alt="La mer de Glace" width="80%">

In more details, the goal is to reproduce the documented retreat as shown on this slide:

<img src="fig/mont_mine_1.png" alt="La mer de Glace" width="80%">
-->

First, we will donwload the data of Mont Miné Glacier stored in folder `RGI60-11.02709` (this can done directly in IGM withing the OGGM module, however, it is but quicker to do it separatly. IGM will notice the data are already there, and won't download through OGGM.)

In [ ]:

import subprocess, os, zipfile

url = "https://www.dropbox.com/scl/fi/29b27a98fchvjgm4fzo99/RGI60-11.02709.zip?rlkey=de8ndc3taa3iw1nraqfidq477&st=3c85sk36&dl=0"
 
if not os.path.exists("RGI60-11.02709"):

    # Download the file with wget
    subprocess.run(['wget', '-O', 'downloaded_file.zip', url]) 

    # Unzipping the file 
    with zipfile.ZipFile('downloaded_file.zip', 'r') as zip_ref:
        zip_ref.extractall('')

    # Clean up (delete) the zip file after extraction
    os.remove('downloaded_file.zip')

## Calibrate the glacier model to match the OSL data

Running `igm_run --param_file params.json` will do a certain number of things :
 - Use 'oggm' to read to collect the data related to the Mont Miné glacier (Switzerland), which as the ID : RGI60-11.02709 (explore https://www.glims.org/maps/glims to find the ID of other glaciers). As said before, having downloaded the data already, this step is in fact ignored, but would be active if we were picking another glacier ID.
 - Run the model forward in time from 1700 to 2024 with a rather simple surface mass balance parametrized by the equilibrium line altitude (ELA).
 - Save the output in a netCDF file `output.nc`.

In [ ]:
! igm_run --param_file params.json

IGM's previous run did the following:
  - Collected data in the folder `RGI60-11.02709` and summarized it in `input_saved.nc`.
  - Reported the time evolution of the ice volume as output from the code.
  - Saved all results in the file `output.nc`.

Let's first take a look at this data. It corresponds to the reconstruction of the ice thickness, surface topography, and surface ice flow field obtained after a "data assimilation" step that is not detailed further here. We can visualize the data by reading `input_saved.nc` using a helper function `plot_input_data` that displays the surface topography , the ice thickness (Millan et al. 2022), and the location of sampling points.

In [ ]:
from helper import *

plot_input_data(file = "input_saved.nc")

Explore `params.json` to see the details of the configuration. A closer look at the parameter file shows that we declare some physical modules, such as `smb_simple`, `iceflow`, `time`, and `thk`.

The first module `smb_simple` computes the surface mass balance as $SMB(z) = \alpha (z - z_{ELA})$, with one SMB gradient $\beta_{abl}$ in the ablation area when $z < z_{ELA}$, and one SMB gradient $\beta_{acc}$ in the accumulation area when $z > z_{ELA}$, along with a maximum SMB capping value of $a_{max}$, as illustrated in the following figure:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def surface_mass_balance(z, z_ELA, beta_abl, beta_acc, a_max):
    smb = np.zeros_like(z)
    smb[z < z_ELA] = beta_abl * (z[z < z_ELA] - z_ELA)
    smb[z > z_ELA] = beta_acc * (z[z > z_ELA] - z_ELA)
    smb[smb > a_max] = a_max
    return smb
 
z = np.linspace(1000, 4000, 100) 
smb_values = surface_mass_balance(z, z_ELA= 3000, beta_abl= 0.005, beta_acc= 0.002, a_max= 1.0)
 
plt.figure(figsize=(10, 6))
plt.plot(z, smb_values, label='Surface Mass Balance (SMB)', color='blue')
plt.axhline(0, color='grey', linestyle='--', linewidth=0.8)  # Line at 0
plt.axvline(3000, color='red', linestyle='--', label='ELA (z_ELA)', linewidth=1)  # ELA line
plt.title('Surface Mass Balance as a Function of Elevation')
plt.xlabel('Elevation (m)')
plt.ylabel('Surface Mass Balance (SMB)') 
plt.legend()
plt.grid()
plt.show()


In `params.json`, the SMB parameters are evolving with time as follows:

```json
  ["time", "gradabl", "gradacc", "ela", "accmax"],
  [ 1700,      0.009,     0.005,  2850,      2.0],
  [ 1800,      0.009,     0.005,  2900,      2.0],
  [ 1850,      0.009,     0.005,  2900,      2.0],
  [ 1900,      0.009,     0.005,  3000,      2.0],  
  [ 2000,      0.009,     0.005,  3100,      2.0],  
  [ 2024,      0.009,     0.005,  3100,      2.0]
```
this allows us to describe a changing ELA to mimic with an advance (to the little ice age), followed by a retreat as the one observed since the end of the little ice age (LIA).

Let's now look at the results, vizualizing the time evolution of the glacier : 

In [ ]:
from helper import *
animate_glacier_evolution('output.nc')

Now, we can visualize the time evolution of the modelled ice thickness at the location of each samples using the following plot.

In [ ]:
from helper import *
plot_thk_at_sample('output.nc')

✅ **Your turn !** 

Modify the parameters in `params.json` as follows (tips: you may use a copy -- e.g. `params_0.json` -- to keep experimenting without losing the original setup):

- Change the SMB parameters: initially adjust the ELA values over time, and later consider modifying the mass balance gradients and maximum accumulation values.
- Set the parameter "iflo_init_slidingco" to a higher value (e.g., 0.4 for less basal sliding) and a lower value (e.g., 0.1 for more basal sliding).

After making these changes, observe how they affect the glacier, and attempt to align the model (by changing iteratively parameters) with the ages of the collected samples.


## To go further : moraine building

In the next step, we will activate particle tracking in the Ice Glacier Model (IGM) to simulate the displacement of virtual particles within the ice flow. These particles may represent debris or sediments that are transported by the glacier and are responsible for the formation of moraines. Our goal is to model the formation of the main moraine that developed during the Little Ice Age when the glacier reached its maximum extent.

To achieve this, we need to make two changes to the parameter file `params.json`. First, we need to add two modules, "vert_flow" and "particles", to the IGM. The "vert_flow" module computes vertical velocities (which is not done by default as it is not usually required), while the "particles" module tracks the trajectory of individual particles. We will accomplish this by adding these modules to the "modules_process" section of the parameter file:

```json
  "modules_process": ["smb_simple",
                      "iceflow",
                      "time",
                      "thk",
                      "vert_flow",
                      "particles"
                      ],
```

Second, one needs to add the following parameters :

```json
  "vflo_method": "kinematic",
  "part_tracking_method": "3d",
  "part_density_seeding": 0.25,
  "part_frequency_seeding": 30
```

It is possible to visualize the moraine formed by looking at the file `velbar_mag-XXXX.0.png`. Compare it with a satellite image of the "[Glacier du Mont Miné](https://www.google.com/maps/place/Mont+Min%C3%A9+Glacier/@46.0145075,7.4877889,24811m/data=!3m1!1e3!4m6!3m5!1s0x478f2db4b5d1a731:0x2c7d28e4d7f7bb26!8m2!3d46.014!4d7.54439!16s%2Fm%2F04n61lb?entry=ttu&g_ep=EgoyMDI0MTAyNy4wIKXMDSoASAFQAw%3D%3D,)"


✅ **Yout turn !** 

Try to adjust the model to reproduce as closely as possible the little ice age moraine.

## To go further : using a **true** Surface Mass Balance Model 

In the next step, we will replace the simple Surface Mass Balance (SMB) model with an improved model known as the [Positive Degree Day Model (PDD)](https://www.antarcticglaciers.org/glaciers-and-climate/numerical-ice-sheet-models/modelling-glacier-melt/). This model computes the SMB based on climate forcing (temperature and precipitation) by seasonally summing the amount of snow precipitation and melt, which is determined as proportional to the number of days with positive temperatures.

To implement this, we need to make two changes to the parameter file `params.json`. First, we will replace the module "smb_simple" with two modules: "clim_oggm" and "smb_oggm," which will update the climate and the surface mass balance, respectively. "clim_oggm" will also take care of retrieving historical climate data located in folder `RGI60-11.02709`, and generate climate forcing frm this.

```json
  "modules_process": ["clim_oggm",
                      "smb_oggm",
                      "iceflow",
                      "time",
                      "thk"
                      ],
```


Second, one needs to replace "smb_simple_array" by another array:

```json
  "clim_oggm_clim_trend_array": [
                        ["time", "delta_temp", "prec_scal"],
                        [ 1700,           0.0,         1.0],
                        [ 1800,           0.0,         1.0],
                        [ 1900,           0.0,         1.0],
                        [ 2024,           1.0,         1.0]
                                 ],
```

Similar to the previous method, this array assigns weights to modify precipitation and temperature series based on a *neutral* climate period, typically defined as 1960-1990. By adjusting the temperature offset in the "delta_temp" column, for example, a value of -2.0 would create a climate scenario for the 1960-1990 period that is 2 degrees cooler. Likewise, by modifying the precipitation scaling in the "prec_scal" column, a value of 1.5 would result in a climate scenario that has 150% of the precipitation compared to the 1960-1990 average.

Finally, you can incorporate this parameter to provide global control for melt (note that the melt parameters in the PDD model are notoriously uncertain):

```json
   "melt_enhancer": 0.7,
```